In [1]:
import smtplib

In [2]:
smtp = smtplib.SMTP('smtp.gmail.com', 587)

In [4]:
#587 port는 메일을 보내는데 사용하는 포트로 사용하겠다

In [9]:
pw='alswn0508!'

In [6]:
smtp.ehlo()

(250,
 b'smtp.gmail.com at your service, [117.111.2.128]\nSIZE 35882577\n8BITMIME\nSTARTTLS\nENHANCEDSTATUSCODES\nPIPELINING\nCHUNKING\nSMTPUTF8')

In [7]:
smtp.starttls() #이제 암호화해서 사용

(220, b'2.0.0 Ready to start TLS')

In [10]:
smtp.login('minju980508', pw)

(235, b'2.7.0 Accepted')

In [11]:
smtp.sendmail('minju980508@gmail.com', ['minju980508@gmail.com', 'minjuyoubin@naver.com'], 'email testing success!!!')

{}

In [12]:
import smtplib
from email.utils import formataddr
from email.mime.text import MIMEText
from email.header import Header
import getpass


smtp_info = {      # SMTP 서버
    'gmail.com': ('smtp.gmail.com', 587),
    'naver.com': ('smtp.naver.com', 587),
    'daum.net': ('smtp.daum.net', 465),
    'hanmail.net': ('smtp.daum.net', 465),
    'nate.com': ('smtp.mail.nate.com', 465),
    'hotmail.com': ('smtp-mail.outlook.com', 587),
    'outlook.com': ('smtp-mail.outlook.com', 587),
    'yahoo.com': ('smtp.mail.yahoo.com', 587),
    }

def send_text_email(sender_tup, receivers, subject, message, passwd):
    '''
    sender_tup: (display_name, email_address)
    receivers: [(display_name, email_address), (display_name, email_address), ...]
    subject: str
    message: str
    passwd: str
    '''
    # 보내는 사람 주소를 'Hong <hong@gmail.com>' 형식으로 변환
    # 예: ('Hong', 'hong@somemail.com') -> 'Hong <hong@gmail.com>'
    sender_form = formataddr(sender_tup)  
    # 수신인들 주소 정리
    mail_to = [formataddr(rec) for rec in receivers]

    # SMPT 서버 선택
    host = sender_tup[1].split('@')[1]  # gmail.com
    smtp_server, port = smtp_info[host] # smtp.gmail.com, 587
    
    # 서버 연결
    if port == 587:
        smtp = smtplib.SMTP(smtp_server, port)
        rcode1, _ = smtp.ehlo()
        rcode2, _ = smtp.starttls()
    else: # if port == 465:
        smtp = smtplib.SMTP_SSL(smtp_server, port)
        rcode1, _ = smtp.ehlo()
        rcode2 = 220
        
    if rcode1 != 250 or rcode2 != 220:
        smtp.quit()
        return 'conection failed'
    
    # 로그인
    rcode3, _ = smtp.login(sender_tup[1], passwd)
    if rcode3 != 235:
        smtp.quit()
        return 'login failed'
    
    # 메일 구성 및 전송
    msg = MIMEText(message.encode('utf-8'), _subtype='plain', _charset='utf-8')
    msg['Subject'] = Header(subject.encode('utf-8'), 'utf-8')
    msg['From'] = sender_form
    msg['To'] = ','.join(mail_to)

    smtp.sendmail(sender_form, mail_to, msg.as_string( ))
    smtp.quit()

In [13]:
myemail = ('김민주', 'minju980508@gmail.com')
receivers = [('김민주', 'minjuyoubin@naver.com'), ('김민주', 'minju980508@gmail.com')]
subject = 'Testing'
message = '''
Testing mail automatically.
This is a mail sending from python.
'''
send_text_email(myemail,receivers, subject, message, pw)

In [14]:
import smtplib
import mimetypes
import os
import chardet   # pip install chardet로 설치

from email.utils import formataddr
from email.header import Header
from email.message import Message
from email.mime.base import MIMEBase
from email.mime.audio import MIMEAudio
from email.mime.image import MIMEImage
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication


smtp_info = {      # SMTP 서버
    'gmail.com': ('smtp.gmail.com', 587),
    'naver.com': ('smtp.naver.com', 587),
    'daum.net': ('smtp.daum.net', 465),
    'hanmail.net': ('smtp.daum.net', 465),
    'nate.com': ('smtp.mail.nate.com', 465),
    'hotmail.com': ('smtp-mail.outlook.com', 587),
    'outlook.com': ('smtp-mail.outlook.com', 587),
    'yahoo.com': ('smtp.mail.yahoo.com', 587),
    }

def send_email(sender_tup, receivers, subject, message, attach_files=(), passwd=''):
    # 보내는 사람 주소 정리
    sender = formataddr(sender_tup)
    # 수신인들 주소 정리
    mail_to = [formataddr(rec) for rec in receivers]

    outer = MIMEBase('multipart', 'mixed')
    outer['Subject'] = Header(subject.encode('utf-8'), 'utf-8')
    outer['From'] = sender
    outer['To'] = ', '.join(mail_to)   # 수신자 문자열 만들기
    outer.preamble = 'This is a multi-part message in MIME format.\n\n'
    outer.epilogue = ''	      # 이렇게 하면 멀티파트 경계 다음에 줄 바꿈 코드가 삽입 됨
    msg = MIMEText(message.encode('utf-8'), _charset='utf-8')
    outer.attach(msg)  # 본문

    for fpath in attach_files:
        folder, file_name = os.path.split(fpath)
        ctype, _ = mimetypes.guess_type(file_name) # 파일 종류 추측
        if ctype is None:
            ctype = 'application/octet-stream'
        maintype, subtype = ctype.split('/', 1)
        with open(fpath, 'rb') as f:
            body = f.read()
        if maintype == 'text':
            encoding = chardet.detect(body)['encoding']
            msg = MIMEText(body, _subtype=subtype, _charset=encoding)
        elif maintype == 'image':
            msg = MIMEImage(body, _subtype=subtype)
        elif maintype == 'audio':
            msg = MIMEAudio(body, _subtype=subtype)
        else:
            msg = MIMEApplication(body, _subtype=subtype)
        msg.add_header('Content-Disposition', 'attachment', 
                       filename=(Header(file_name, 'utf-8').encode()))
        outer.attach(msg) # 파일 첨부

    # SMTP 서버를 통해서 메일 보내기
    _, host = sender_tup[1].rsplit('@', 1)
    smtp_server, port = smtp_info[host]
    
    if port == 587:
        smtp = smtplib.SMTP(smtp_server, port)
        rcode1, _ = smtp.ehlo()
        rcode2, _ = smtp.starttls()
    else: # if port == 465:
        smtp = smtplib.SMTP_SSL(smtp_server, port)
        rcode1, _ = smtp.ehlo()
        rcode2 = 220
        
    if rcode1 != 250 or rcode2 != 220:
        smtp.quit()
        return 'conection failed'
        
    smtp.login(sender_tup[1], passwd)
    smtp.sendmail(sender, mail_to, outer.as_string())
    smtp.quit()

In [16]:
!pip install chardet

In [18]:
myemail = ('김민주', 'minju980508@gmail.com')
receivers = [('김민주', 'minjuyoubin@naver.com'), ('김민주', 'minju980508@gmail.com')]
subject = 'Testing'
message = '''
Testing mail with file automatically.
This is a mail sending from python.
'''
attach_file = [r'C:\Users\minju\Anaconda3\source\1.png']
send_email(myemail, receivers, subject, message, attach_file, pw)